## Zaawansowane Metody Inteligencji Obliczeniowej
# Zadanie domowe 1
### Prowadzący: Michał Kempka, Marek Wydmuch
### Autor: Daniel Zdancewicz Indeks 145317

## Wprowadzenie

Całe zadanie jest oparte o różne wersje środowiska `VacuumEnvironemnt`, które rozważaliśmy na zajęciach.
Środowisko zaimplementowane jest w bibliotece aima3 (https://github.com/ArtificialIntelligenceToolkit/aima3),
która zawiera kod do książki "Artificial Intelligence: A Modern Approach".

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod) o ile nie napisano gdzieś inaczej.

In [1]:
# Zainstaluj bibliotekę OpenAI Gym
!pip install aima3


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import collections
collections.Callable = collections.abc.Callable

# Zaimportuj wszystkie jego elementy
from aima3.agents import *

Wszystkie używane przez nas elementy biblioteki są zaimplementowane w pliku: https://github.com/ArtificialIntelligenceToolkit/aima3/blob/master/aima3/agents.py

# Zad. 1 - Cechy środowiska odkurzacza (1 pkt.)

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź:
- Obserwowalność: Częściowo obserwowalne — stan pokoju jest widoczny dla agenta, ale nie jest widoczny drugi.
- Liczność agentów: Środowisko Jednoagentowe — jest tylko jeden odkurzacz
- Determinizm: Deterministyczne — Wiemy co następuje w następnym kroku po każdej z akcji.
- Statyczność: Środowisko statyczne — nie ma zmian w środowisku poza akcjami agenta i jest na zasadzie tur.
- Dyskretność/Ciągłość czasu: Dyskretny — czas jest skokowy (turami).
- Dyskretność/Ciągłość akcji: Dyskretne — Akcje są opisane przez zbiór o skończonej liczbie elementów { 'Right', 'Left', 'Suck', 'NoOP' }.
- Dyskretność/Ciągłość przestrzeni stanów: Dyskretne — środowisko jest dyskretnie opisane przez stany czystości i lokacji agenta i może być opisane w 8 sposobów kombinatorycznie.
- Model środowiska jest znany.

Tip: Możesz sprawdź implementacje środowiska w pliku podanym powyżej, lub wywnioskować cechy na wykonując poniższe fragmenty kodu.

In [3]:
# Stwórz nowe środowisko świata odkurzacza
env = TrivialVacuumEnvironment()

In [4]:
# Sprawdź aktualny status środowiska
env.status

{(0, 0): 'Dirty', (1, 0): 'Clean'}

In [5]:
# Utwórz agenta refleksyjnego
agent = ReflexVacuumAgent()
agent.is_alive()

True

In [6]:
# Dodaj agenta do środowiska. Owijamy go w TraceAgent'a, żeby zobaczyć co robi.
env.add_thing(TraceAgent(agent))

In [7]:
# Zobacz gdzie jest agent
for loc in [loc_A, loc_B]:
  print('loc {0}: {1}'.format(loc, env.list_things_at(loc)))
# Lub:
agent.location

loc (0, 0): []
loc (1, 0): [<Agent>]


(1, 0)

In [8]:
# Wykonaj 10 kroków
env.run(10)

<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Dirty') and does Suck
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left


In [9]:
# Sprawdź jak środowisko oceniło jakość agenta.
agent.performance

1

In [10]:
# Moglibyśmy ocenić oczekiwaną jakość agenta dokładniej..., ale tylko ją oszacujemy (1000 powtórzeń).
# Zakładamy, że symulacja trwa 50 kroków.

compare_agents(TrivialVacuumEnvironment, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent()>, -39.011),
 (<function aima3.agents.ModelBasedVacuumAgent()>, 8.99)]

# Zad. 2 - Cechy zmodyfikowanego środowisko odkurzacza (1 pkt).

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź:
- Obserwowalność: Częściowo obserwowalne — stan pokoju jest widoczny dla agenta, ale nie jest widoczny drugi.
- Liczność agentów: Środowisko Jednoagentowe — jest tylko jeden odkurzacz
- Determinizm: Stochastyczne — Nie mamy pewności, co następuje po wykonaniu akcji.
- Statyczność: Środowisko statyczne — nie ma zmian w środowisku poza akcjami agenta i jest na zasadzie tur.
- Dyskretność/Ciągłość czasu: Dyskretny — czas jest skokowy (turami).
- Dyskretność/Ciągłość akcji: Dyskretne — Akcje są opisane przez zbiór o skończonej liczbie elementów { 'Right', 'Left', 'Suck', 'NoOP' }.
- Dyskretność/Ciągłość przestrzeni stanów: Dyskretne — środowisko jest dyskretnie opisane przez stany czystości i lokacji agenta i może być opisane w 8 sposobów kombinatorycznie.
- Model środowiska jest znany — znamy prawdopodobieństwa elementów stochastycznych.


In [11]:
# Rozszerzmy implementacje TrivialVacuumEnvironment

import random

class TrivialVacuumEnvironmentWithCats(TrivialVacuumEnvironment):
  def __init__(self, random_dirt_prob=0.05, seed=None):
    super(TrivialVacuumEnvironmentWithCats, self).__init__()
    self.random = random.Random(seed)
    self.random_dirt_prob = random_dirt_prob

  def execute_action(self, agent, action):
    """Change agent's location and/or location's status; track performance; add dirt;
    Score 10 for each dirt cleaned; -1 for each move."""
    # Same as in case of TrivialVacuumEnvironment
    if action == 'Right':
      agent.location = loc_B
      agent.performance -= 1
    elif action == 'Left':
      agent.location = loc_A
      agent.performance -= 1
    elif action == 'Suck':
      if self.status[agent.location] == 'Dirty':
        agent.performance += 10
      self.status[agent.location] = 'Clean'

    # Cats can make either location dirty
    for loc in [loc_A, loc_B]:
      if self.random.random() < self.random_dirt_prob:
        self.status[loc] = 'Dirty'

In [12]:
# Przetestujmy domyślnych agentów w nowym środowisku

def env_factory():
  return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)

compare_agents(env_factory, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent()>, 11.831),
 (<function aima3.agents.ModelBasedVacuumAgent()>, 32.88)]

# Zad. 3 - Własny program agenta (8 pkt.)

Napisz program agenta, który będzie (średnio) dużo lepszy dla tego środowiska (50 kroków, z random_dirt_prob=0.05) niż ModelBaseVacuumAgent oraz ReflexVacuumAgent. Opisz działanie swojego programu, na podstawie jaki przesłanek on działa, jakbyś go zmodyfikował gdyby prawdopodobieństwo zabrudzenia pokoju (random_dirt_prob) się zmieniło?

Punktacja za wynik (sprawdzarka zrobi 50000 powtórzeń):
* \> 41: 1 pkt.
* \> 42: 2 pkt.
* \> 43: 3 pkt.
* \> 44: 4 pkt.
* \> 45: 5 pkt.
* \> 46: 6 pkt.

\+ 2 pkt. za opis.

#### Uwaga: nie zmieniaj nazwy klasy `MyVacuumAgent`. Nie dopisuj do komórki z klasą innego kodu. Możesz zdefiniować funkcje pomocnicze w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórkę z klasę o nazwie `MyVacuumAgent` do sprawdzenia).

In [248]:
from typing import Literal
# Klasa MyVacuumAgent wypełniona przykładowym kodem agenta z modelem

DirtChance = 0.05
MoveLimit = 50
create_memory = lambda: {
  "moves_left": MoveLimit,
  "last_move": MoveLimit,
  "tolerance": 0.40
}
Location = tuple[int, int]
Status = Literal['Clean', 'Dirty']
Action = Literal['Suck', 'Right', 'Left', 'NoOP']
Noop = 'NoOP'
Right = 'Right'
Left = 'Left'
Suck = 'Suck'
Dirty = 'Dirty'

def MyVacuumAgent():
  def calculate_dirt_chance():
    return 1 - (1 - DirtChance) ** memory["last_move"]

  def consider_move(perception: tuple[Location, Status]) -> Action:
    memory["moves_left"] -= 1
    memory["last_move"] += 1
    dirt_chance = calculate_dirt_chance()

    (location, status) = perception
    if status == Dirty or memory["moves_left"] < 1: return Suck
    if dirt_chance > memory["tolerance"] or memory["moves_left"] == 1 and dirt_chance > memory["tolerance"] / 2:
      memory["last_move"] = 0
      return location == loc_A and Right or Left
    return Suck
  memory = create_memory()
  return Agent(consider_move)

Opis:
Agent ma pamięć, która pozwala mu na obliczanie prawdopodobieństwa wystąpienia przynajmniej jednej generacji, w której koty przyniosły brud.
Pamięć jest aktualizowana na początku każdej tury.
Prawdopodobieństwo wystąpienia brudu jest obliczane na podstawie wzoru opisującej dystrybuantę binomialną. Tolerancja była obliczana eksperymentalnie, na początku wychodziło, że jest najlepiej, gdy 0.25, potem 0.35 i ostatecznie wyszło, że jednak 0.40 jest najlepsze.
Dodałem też early return, gdy agent ma tylko jeden ruch, bo wtedy nie ma sensu myśleć, trzeba ssać, bo i tak nie ruszymy się do innego pokoju.
Domyślnie też robot ssie, bo nie ma za to kary i przez to jest potencjalnie lepszą akcją od zwykłego czekania.
Jeżeli prawdopodobieństwo przewaga tolerancję bota to ten rusza się do innego pokoju, no, a jeżeli nie to ssie.
Jeszcze jest oportunistyczny ostatni ruch o zmniejszonej tolerancji, bo możemy złapać 10 punktów kosztem potencjalnie 1 i to dało tą ostateczną przewagę by dociągnąć do >46. Bez tego ostatniego ruchu byłoby ciągle w okolicach 45.8.


In [249]:
compare_agents(
  lambda: TrivialVacuumEnvironmentWithCats(random_dirt_prob=DirtChance),
  [MyVacuumAgent], 50000, MoveLimit
)

[(<function __main__.MyVacuumAgent()>, 46.62402)]

In [24]:

# best_reward = 0
# best_tolerance = 0
# for i in range(100 + 1):
#   print(f"{i + 1}/100")
#   tolerance = i / 100
#   [(_, reward)] = compare_agents(
#     lambda: TrivialVacuumEnvironmentWithCats(random_dirt_prob=DirtChance),
#     [lambda: MyVacuumAgent(tolerance)], 10000, MoveLimit
#   )
#   if (reward > best_reward):
#     best_tolerance = tolerance
#     best_reward = reward
# print(f"{best_tolerance=} {best_reward=}")
compare_agents(
  lambda: TrivialVacuumEnvironmentWithCats(random_dirt_prob=DirtChance),
  [MyVacuumAgent], 2000, MoveLimit
)


[(<function __main__.MyVacuumAgent()>, 41.752)]